# TM10007 Assignment template

In [1]:
# Run this to use from colab environment
!git clone https://github.com/Doesjka/TM10007_ML_g9.git

Cloning into 'TM10007_ML_g9'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 118 (delta 31), reused 17 (delta 17), pack-reused 72
Receiving objects: 100% (118/118), 68.30 MiB | 21.89 MiB/s, done.
Resolving deltas: 100% (48/48), done.


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import decomposition
import seaborn
import matplotlib.pyplot as plt

data = pd.read_csv('/content/TM10007_ML_g9/worclipo/Lipo_radiomicFeatures.csv', index_col=0)
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 115
The number of columns: 494


## Splitting data in train and test set


In [3]:
y = data['label']
X = data.drop('label', axis=1)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)


## Handling missing data 
### Throwing out features
All features that exist of at least 50% zeros are deleted from the data. 


In [4]:
zeros = (X_train == 0).sum()
threshold = 0.5 * len(y_train)
print('Threshold = ', threshold)
feature_del = zeros[zeros > threshold]

X_train = X_train.drop(columns=feature_del.index)
print(f'{len(data.columns)-len(X_train.columns)} features were deleted')

Threshold =  43.0
25 features were deleted


In [5]:
more_zeros = (X_train == 0).sum()
columns_zeros = more_zeros[more_zeros > 0].index
print(f'Of the remaining features, {len(columns_zeros)} features have at least one zero')
print(f'There is a total of {more_zeros.sum()} zeros left in the data')


Of the remaining features, 10 features have at least one zero
There is a total of 44 zeros left in the data


## Calculate number of missing values per sample

In [6]:
zeros_r = (X_train == 0).sum(axis=1)
threshold = 0.005 * X_train.size / len(y_train)
print('Threshold = ', threshold)
feature_del = zeros_r[zeros_r > threshold]
print(feature_del)

Threshold =  2.345
ID
Lipo-090_0    5
Lipo-095_0    3
Lipo-076_0    3
Lipo-003_0    3
dtype: int64


In [7]:
from scipy.stats import shapiro

aantal_normaal = 0

for column in X_train.columns:
    result = shapiro(X_train[column])
    # print(result.pvalue)
    normaal = result.pvalue > 0.05
    aantal_normaal += normaal
    
print(aantal_normaal)



/usr/local/lib/python3.9/dist-packages/scipy/stats/_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


74


### Filling remaining zeros
All remaining zeros are replaced by the mean of that feature. 

In [8]:
X_train_mean = X_train
X_train_median = X_train

for column in columns_zeros[:]:
    print('Kolom: ', column)
    column_mean = X_train.loc[X_train[column]!=0, column].mean()
    column_median = X_train.loc[X_train[column]!=0, column].median()
    print('mean = ', column_mean)
    print('median = ', column_median)

    verschil2_percolumn = column_mean - column_median
    print('Verschil tussen mean en median = ',verschil2_percolumn)

    # result = shapiro(X_train[column])
    # result.pvalue
    print('p-waarde normaalverdeling = ', shapiro(X_train.loc[X_train[column]!=0, column]).pvalue)

    X_train_mean[column].replace(0, column_mean)
    X_train_median[column].replace(0, column_median)

    print(' ')

verschil = abs(X_train_mean - X_train_median)
print('Totaal verschil = ', verschil.sum().sum())

# X_train.head()
# pd.set_option('display.max_rows', None)
# print(X_train[columns_zeros[0]])
# pd.set_option('display.max_rows', 10)


Kolom:  PREDICT_original_sf_area_min_2.5D
mean =  456.8698899846219
median =  200.39520811289555
Verschil tussen mean en median =  256.4746818717264
p-waarde normaalverdeling =  3.05788068477586e-11
 
Kolom:  PREDICT_original_tf_LBP_quartile_range_R8_P24
mean =  9.301204819277109
median =  11.0
Verschil tussen mean en median =  -1.6987951807228914
p-waarde normaalverdeling =  1.931198130478151e-05
 
Kolom:  PREDICT_original_vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0
mean =  2.766293780856362e-07
median =  4.602205232532876e-10
Verschil tussen mean en median =  2.7616915756238295e-07
p-waarde normaalverdeling =  9.707482768662333e-20
 
Kolom:  PREDICT_original_vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0
mean =  2.766293780856362e-07
median =  4.602205232532876e-10
Verschil tussen mean en median =  2.7616915756238295e-07
p-waarde normaalverdeling =  9.707482768662333e-20
 
Kolom:  PREDICT_original_vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0
mean =  8.705055316112096e-

## Scaling

In [9]:
scaler = preprocessing.StandardScaler().fit(X_train_median)
X_train_scaled = scaler.transform(X_train_median)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_median.columns)

# scaler = preprocessing.RobustScaler().fit(X_train_median)
# X_train_scaled = scaler.transform(X_train_median)
# X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_median.columns)

# scaler = preprocessing.MinMaxScaler().fit(X_train_median)
# X_train_scaled = scaler.transform(X_train_median)
# X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_median.columns)

In [10]:
import scipy.stats as stats
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 10)

# stats.zscore(X_train)    # Dit is voor een array 
Z_score = X_train.apply(stats.zscore)    # Dit is voor een pandas dataframe 

num_outliers = (np.abs(Z_score) > 3).sum()
outliers = num_outliers.sum()
numb_columns_outliers = (num_outliers > 0).sum()

print(num_outliers)
print(numb_columns_outliers)
print(outliers)

PREDICT_original_sf_compactness_avg_2.5D                  0
PREDICT_original_sf_compactness_std_2.5D                  0
PREDICT_original_sf_rad_dist_avg_2.5D                     0
PREDICT_original_sf_rad_dist_std_2.5D                     1
PREDICT_original_sf_roughness_avg_2.5D                    2
                                                         ..
PREDICT_original_phasef_phasesym_kurtosis_WL3_N5          1
PREDICT_original_phasef_phasesym_range_WL3_N5             0
PREDICT_original_phasef_phasesym_energy_WL3_N5            3
PREDICT_original_phasef_phasesym_quartile_range_WL3_N5    0
PREDICT_original_phasef_phasesym_entropy_WL3_N5           0
Length: 469, dtype: int64
333
601


## PCA

In [48]:
pca = decomposition.PCA()
pca.fit(X_train_scaled_df)
X_pca = pca.transform(X_train_scaled_df)

component=0
total_ratio = 0
while total_ratio < 0.9:
    total_ratio += pca.explained_variance_ratio_[component]
    component+=1

X_pca = X_pca[:,0:component]

28


## k-NN

In [55]:
from sklearn import neighbors
# Specify the search range, this could be multiple parameters for more complex classifiers
parameters = {'n_neighbors': randint(1, 50),
              'weights': ['uniform', 'distance'],
              'p': randint(1, 5),
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

# Specify the cross validation method to use, we use 10-fold stratified cross-validation
cv_10fold = model_selection.StratifiedKFold(n_splits=10)

# Create the grid search method, use area under ROC curve as scoring metric
# Too learn more about metrics see: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
clf = model_selection.RandomizedSearchCV(neighbors.KNeighborsClassifier(), parameters, cv=cv_10fold, n_iter=500, scoring='accuracy')

# Do the entire search
clf.fit(X_pca, y_train)

# Show the complete results of the cross validation
clf_df = pd.DataFrame(clf.cv_results_)

# Extract the best k 
clf_df = clf_df.sort_values(by=['rank_test_score'])
optimal_k = clf_df['param_n_neighbors'].iloc[0]

# Extract the best hyperparameters 
print(clf.best_params_)
print(clf.best_score_)



{'algorithm': 'ball_tree', 'n_neighbors': 31, 'p': 1, 'weights': 'distance'}
0.6611111111111111


## Random Forest

In [66]:
from scipy.stats import randint
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

# Create the random grid
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)

parameters = {'n_estimators': np.arange(10, 400, 10),
              'max_depth': [None, 5, 10, 15],
              'min_samples_split': [2, 5, 10],
              'max_features': [None, 'sqrt', 'log2'],
              'min_samples_leaf': [1, 2, 4]}

cv_10fold = model_selection.StratifiedKFold(n_splits=10)

clf = model_selection.RandomizedSearchCV(RandomForestClassifier(), parameters, cv=cv_10fold, n_iter=50, scoring='accuracy')
    
# Fit the classifier
clf.fit(X_pca, y_train)

# Show the complete results of the cross validation
clf_df = pd.DataFrame(clf.cv_results_)
clf_df = clf_df.sort_values(by=['rank_test_score'])

# Extract the best hyperparameters 
print(clf.best_params_)
print(clf.best_score_)


{'n_estimators': 340, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}
0.6583333333333333


## SVM

In [59]:
import scipy 
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.utils.fixes import loguniform

# parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
#              {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}]

parameters = {'C': loguniform(0.1, 100),
              'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
              'degree': randint(1, 5),
              'gamma': loguniform(1e-4, 1),
              'class_weight':['balanced', None]}
 
cv_10fold = model_selection.StratifiedKFold(n_splits=10)

clf = model_selection.RandomizedSearchCV(SVC(), parameters, cv=cv_10fold, n_iter=500, scoring='accuracy')

# Do the entire search
clf.fit(X_pca, y_train)

# Show the complete results of the cross validation
clf_df = pd.DataFrame(clf.cv_results_)
clf_df = clf_df.sort_values(by=['rank_test_score'])

# Extract the best hyperparameters 
print(clf.best_score_)
print(clf.best_params_)

0.7694444444444445
{'C': 3.4392592585305524, 'class_weight': 'balanced', 'degree': 1, 'gamma': 0.01143799983523973, 'kernel': 'poly'}
